In [1]:
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel

f:\test\chatbot\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

f:\test\chatbot\env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open("../data/images/005.jpg")

# Các mô tả tình huống nguy hiểm có thể gặp trong nuôi trồng thủy sản
descriptions = [
  "Độ pH không phù hợp",
  "Nhiệt độ nước không phù hợp",
  "Oxy hòa tan thấp",
  "Độ mặn không phù hợp trong ao nuôi",
  "Amoniac tích tụ",
  "Nitrit gây độc hại",
  "Độ kiềm không phù hợp trong ao nuôi",
  "Chất rắn lơ lửng tăng",
  "Nồng độ H2S cao",
  "Độ trong thấp",
  "Chỉ số oxy hóa khử thấp",
  "Cường độ ánh sáng thấp",
  "Tảo độc hại",
  "Có vi sinh vật gây bệnh"
]

inputs = processor(text=descriptions, images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities
 
# In xác suất tương đồng
for i, description in enumerate(descriptions):
  print(f"Description: {description}, Probability: {probs[0][i].item():.4f}")

Description: Độ pH không phù hợp, Probability: 0.0241
Description: Nhiệt độ nước không phù hợp, Probability: 0.0839
Description: Oxy hòa tan thấp, Probability: 0.1283
Description: Độ mặn không phù hợp trong ao nuôi, Probability: 0.1772
Description: Amoniac tích tụ, Probability: 0.0639
Description: Nitrit gây độc hại, Probability: 0.0278
Description: Độ kiềm không phù hợp trong ao nuôi, Probability: 0.1187
Description: Chất rắn lơ lửng tăng, Probability: 0.0797
Description: Nồng độ H2S cao, Probability: 0.0017
Description: Độ trong thấp, Probability: 0.0415
Description: Chỉ số oxy hóa khử thấp, Probability: 0.0224
Description: Cường độ ánh sáng thấp, Probability: 0.0903
Description: Tảo độc hại, Probability: 0.0690
Description: Có vi sinh vật gây bệnh, Probability: 0.0714


In [17]:
# Select alarm thresholds and status notifications
threshold = 0.6  # Ngưỡng để kích hoạt cảnh báo
alert_triggered = False

for i, description in enumerate(descriptions):
  if probs[0][i].item() > threshold and description != "ao nuôi bình thường":
    print(f"Cảnh báo: {description} phát hiện với xác suất {probs[0][i].item():.4f}")
    alert_triggered = True

if not alert_triggered:
  print("Tình trạng bình thường, không có cảnh báo.")

Tình trạng bình thường, không có cảnh báo.
